In [68]:
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import numpy as np

pic_size = 48                         


base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# Function to preprocess images
def preprocess_image(img_path, target_size=(48, 48)):
    """Load and preprocess an image."""
    img = load_img(img_path, target_size=target_size, color_mode="grayscale")  # Grayscale for custom CNNs
    img_array = img_to_array(img)  # Convert to NumPy array
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Function to load and preprocess dataset
def load_dataset(base_path, subset='train'):
    """Load and preprocess the dataset."""
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue  

        for img_name in os.listdir(expression_path):
            img_path = os.path.join(expression_path, img_name)
            try:
                img_array = preprocess_image(img_path)
                images.append(img_array)
                labels.append(label_map[expression])  
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    return np.array(images), np.array(labels)



# Function to visualize sample images
def visualize_samples(base_path, subset='train'):
    """Visualize sample images for each expression."""
    plt.figure(figsize=(12, 20))           
    counter = 0                            
    subset_path = os.path.join(base_path, subset)
    
    for expression in os.listdir(subset_path):                      
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue                                                
        
        for i, img_name in enumerate(os.listdir(expression_path)[:5]):  
            counter += 1                                               
            plt.subplot(7, 5, counter)                                  
            img_path = os.path.join(expression_path, img_name)
            img_array = preprocess_image(img_path)
            plt.imshow(img_array, cmap="gray")                         
            plt.xlabel(expression)
    
    plt.tight_layout()
    plt.show()


# Function to count images per expression
def count_images_per_expression(base_path, subset='train'):
    """Count and display the number of images per expression."""
    subset_path = os.path.join(base_path, subset)
    for expression in os.listdir(subset_path):
        expression_path = os.path.join(subset_path, expression)
        if not os.path.isdir(expression_path):
            continue
        
        num_images = len(os.listdir(expression_path))
        print(f"{num_images} {expression} images")


In [69]:
count_images_per_expression(base_path, subset='train')
##visualize_samples(base_path, subset='train')

7164 happy images
4938 sad images
4103 fear images
3205 surprise images
4982 neutral images
3993 angry images
436 disgust images


In [70]:
count_images_per_expression(base_path, subset='validation')
#visualize_samples(base_path, subset='validation')

1825 happy images
1139 sad images
1018 fear images
797 surprise images
1216 neutral images
960 angry images
111 disgust images


In [71]:
#### Error handling:---- don't run now
#for img_name in os.listdir(expression_path):
   # img_path = os.path.join(expression_path, img_name)
   # try:
    #    img_array = preprocess_image(img_path)
     #   images.append(img_array)
     #   labels.append(label_map[expression])
   # except Exception as e:
   #     print(f"Error loading image {img_name}: {e}")


In [72]:
train_images, train_labels = load_dataset(base_path, subset='train')
test_images, test_labels = load_dataset(base_path, subset='validation')

print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Training images shape: (28821, 48, 48, 1)
Training labels shape: (28821,)
Test images shape: (7066, 48, 48, 1)
Test labels shape: (7066,)


-----------------------------------------------------------------------------------

In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                                                       


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

batch_size = 64   

train_datagen = ImageDataGenerator(
    rescale=1.0/255,             
    rotation_range=10,            
    width_shift_range=0.1,        
    height_shift_range=0.1,               
    zoom_range=0.1,               
    horizontal_flip=True,        
    fill_mode='nearest'           
)

      
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [83]:

train_generator = train_datagen.flow_from_directory(base_path + "/train",           
                                                    target_size=(pic_size,pic_size), 
                                                    color_mode="grayscale",           
                                                    batch_size=batch_size,            
                                                    class_mode='categorical',         
                                                    shuffle=True)                    
validation_generator = validation_datagen.flow_from_directory(base_path + "/validation",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [85]:
class_labels = train_generator.class_indices
class_weights = compute_class_weight(
    'balanced', 
    classes=np.array(list(class_labels.values())), 
    y=train_generator.classes
)

# Create a dictionary of class weights
class_weight_dict = dict(zip(class_labels.keys(), class_weights))

# Print the class weights
print("Class weights:", class_weight_dict)

Class weights: {'angry': 1.031125898894494, 'disgust': 9.443315858453474, 'fear': 1.0034817729187702, 'happy': 0.5747188322565207, 'neutral': 0.8264322991340254, 'sad': 0.8337962159347335, 'surprise': 1.2846445286382884}


-----------------------------------------------------------------------------------

In [86]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# Number of labels :
num_labels =  7

In [88]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Sequential()
# 1st Convolution layer
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.25)) 

# 2nd Convolution layer
model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  

In [89]:
opt = Adam(learning_rate=0.0001)                                                                 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])   

In [90]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_44 (Conv2D)              │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_40 (Activation)      │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_41 (Activation)      │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_42 (Activation)      │ (None, 12, 12, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_43 (Activation)      │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_51 (Dropout)            │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,227,143 (12.31 MB)

 Trainable params: 3,224,711 (12.30 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [92]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")

Steps per epoch: 450, Validation steps: 110


In [93]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(
    "model_weights.keras",  
    monitor='val_accuracy',  
    verbose=1,  
    save_best_only=True,  
    mode='max'   
)
callbacks_list = [checkpoint]


early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    x=train_generator,  # Training data
    epochs=20,  # Start with 5 epochs for a quick test
    validation_data=validation_generator,  # Validation data
    steps_per_epoch = len(train_generator),
    validation_steps = len(validation_generator),
    batch_size=64,  # Batch size
    class_weight=class_weight_dict,  # Add class weights
    callbacks=[checkpoint, early_stop]  # Include both checkpoint and early stopping
)

Epoch 1/20


/Users/pratiksha/.pyenv/versions/3.9.19/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 818ms/step - accuracy: 0.2149 - loss: 2.9471
Epoch 1: val_accuracy improved from -inf to 0.17223, saving model to model_weights.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 389s 859ms/step - accuracy: 0.2149 - loss: 2.9469 - val_accuracy: 0.1722 - val_loss: 2.9081
Epoch 2/20


2024-11-25 19:35:09.372529: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/pratiksha/.pyenv/versions/3.9.19/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-25 19:35:09.391093: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

# Plot Loss
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer: Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()


In [ ]:
# Show the confusion matrix of our predictions

# Compute predictions
predictions = model.predict(validation_generator)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()


In [ ]:

model.save('/Users/pratiksha/Documents/Pratiksha/Documents/GitHub/GitHub/Face-expression-recognition-with-Deep-Learning/my_model.keras')
